![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Загружаем пакет для работы со временем:
from datetime import datetime, timedelta

# Загружаем модель для работы с регулярными выражениями для чистки данных
import re

# Загружаем модуль collections для подсчета частоты встречаемости уникальных значений
import collections

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
# Загружаем данные
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [2]:
data = pd.read_csv('main_task.csv')

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

# Загрузим и посмотрим на весь датасет

In [ ]:
display(data.info())
display(data.describe())
display(data.describe(include = ['object']))

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

In [3]:
# приведем наименование некоторых столбцов к более удобному виду, уберем пробелы
data = data.rename(columns={'Cuisine Style': 'Cuisine_Style','Price Range': 'Price_Range', 'Number of Reviews': 
                            'Number_of_Reviews'})

# Рассмотрим все 10 столбцов по отдельности, выполним очистку и преобразование данных, выполним EDA, создадим новые признаки для модели

### Restaurant_id — идентификационный номер ресторана / сети ресторанов

In [ ]:
data.loc[:, ['Restaurant_id']].info()
display(pd.DataFrame(data.Restaurant_id.value_counts()))
data.Restaurant_id.describe()

##### нет пропусков, категориальный признак, ID повторяются достаточно часто, предположим, что это сети ресторанов 

In [4]:
# формируем соотвествующий признак
# "0" - Restaurant_id - уникальный
# "1" - Restaurant_id повторяется (предположим, это сеть)
chain = data.Restaurant_id.value_counts()[data.Restaurant_id.value_counts()>1].index.tolist()
data['Сhain'] = data[data.Restaurant_id.isin(chain)].Restaurant_id.apply(lambda x: 1)
data.Сhain = data['Сhain'].fillna(0)

In [6]:
# познакомимся с новым столбцом
data.loc[:, ['Сhain']].info()
display(pd.DataFrame(data.Сhain.value_counts()))
data.Сhain.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Сhain   40000 non-null  float64
dtypes: float64(1)
memory usage: 312.6 KB


,Сhain
1.0,35295
0.0,4705


count    40000.000000
mean         0.882375
std          0.322168
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Сhain, dtype: float64

### City — город, в котором находится ресторан

In [ ]:
data.loc[:, ['City']].info()
display(pd.DataFrame(data.City.value_counts()))
data.City.describe()

#### нет пропусков, категориальный признак, всего 31 город, преобразуем методом get_dummies, подход - One-Hot Encoding, исходную колонку пока сохраним

In [5]:
# Создадим дамми переменные для каждого города
data['City_gd'] = data['City']
data = pd.get_dummies(data, columns=[ 'City_gd',], dummy_na=True)
# так как пропущенных значений нет столбец City_gd_nan удалим
data = data.drop(['City_gd_nan'], axis = 1)

In [6]:
# создаем признак 'City_Num_rest',который содержит информацию о количестве заведений в городе

cities1 = {'London':5757,
'Paris':4897,
'Madrid':3108,
'Barcelona':2734,
'Berlin':2155,
'Milan':2133,
'Rome':2078,
'Prague':1443,
'Lisbon':1300,
'Vienna':1166,
'Amsterdam':1086,
'Brussels':1060,
'Hamburg':949,
'Munich':893,
'Lyon':892,
'Stockholm':820,
'Budapest':816,
'Warsaw':727,
'Dublin':673,
'Copenhagen':659,
'Athens':628,
'Edinburgh':596,
'Zurich':538,
'Oporto':513,
'Geneva':481,
'Krakow':443,
'Oslo':385,
'Helsinki':376,
'Bratislava':301,
'Luxembourg':210,
'Ljubljana':183}

data['City_Num_rest'] = data['City'].replace(cities1, inplace=True)

In [8]:
# создаем признак 'City_Population', который содержит информацию о численности населения города и агломерации
# (взято с wiki)

cities2 = {'London':8982000,
'Paris':2196936,
'Madrid':6642000,
'Barcelona':5575000,
'Berlin':3769000,
'Milan':1352000,
'Rome':2873000,
'Prague':1309000,
'Lisbon':504718,
'Vienna':1897000,
'Amsterdam':821752,
'Brussels':174383,
'Hamburg':1845000,
'Munich':1472000,
'Lyon':516092,
'Stockholm':975904,
'Budapest':1752000,
'Warsaw':1708000,
'Dublin':1388000,
'Copenhagen':602481,
'Athens':664046,
'Edinburgh':482005,
'Zurich':402762,
'Oporto':214349,
'Geneva':499480,
'Krakow':779115,
'Oslo':681067,
'Helsinki':631695,
'Bratislava':424428,
'Luxembourg':613894,
'Ljubljana':279631}

data['City_Population'] = data['City'].replace(cities2, inplace=True)

In [7]:
# создаем признак 'City_Turists',который содержит информацию о количестве туристов посетивших город по данным Mastercard
# основной источник: https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors

cities3 = {'Luxembourg':3000000, 
'Zurich':2240000, 
'Geneva':1150000, 
'Dublin':4970000, 
'Oslo':4868681, 
'Amsterdam':8000000, 
'Copenhagen':1630000, 
'Stockholm':2080000, 
'Vienna':6690000, 
'Helsinki':4000000, 
'Berlin':4940000, 
'Hamburg':1450000, 
'Munich':5250000, 
'Brussels':2710000, 
'London':19880000, 
'Paris':18030000, 
'Lyon':4868539, 
'Milan':7650000, 
'Rome':7120000, 
'Madrid':5260000, 
'Barcelona':8200000, 
'Ljubljana':841320, 
'Prague':5810000, 
'Lisbon':3630000, 
'Oporto':4868539, 
'Athens':2680000, 
'Bratislava':1400000, 
'Budapest':3360000, 
'Warsaw':1370000, 
'Krakow':1313277, 
'Edinburgh':1660000}

data['City_Turists'] = data['City'].replace(cities3, inplace=True)

In [9]:
# создаем признаки 'Rest_on_Pop' и 'Rest_on_Turists',
# которые содержыт информацию о количестве ресторанов на одного жителя и туриста

data['Rest_on_Pop'] = data['City_Num_rest']/data['City_Population']
data['Rest_on_Turists'] = data['City_Num_rest']/data['City_Turists']

### Cuisine_Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане

In [ ]:
data.loc[:, ['Cuisine_Style']].info()
display(pd.DataFrame(data.Cuisine_Style.value_counts()))
data.Cuisine_Style.describe()

#### есть пропуски, категориальный признак, необходима обработка

In [10]:
# создаем признак 'Number_of_Reviews_isNAN',который содержит инфо о пропусках
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine_Style']).astype('uint8')

In [ ]:
# судя по формату данных необходимо выполнить очистку от [], '',
# приведем к одному формату (lower, без пробелов, то есть , одна кухня - одно слово),
# разделим виды кухонь, разделитель ',' и преобразуем в список
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: str(x))
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace("[",""))
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace("]",""))
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace("'",""))
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace(" ",""))
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.lower())
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.split(","))

In [12]:
# Создаем словарь ключь - кухня, значение - количество ресторанов с этой кухней
cuisine_exploded = data.explode('Cuisine_Style')
cousins = dict(cuisine_exploded['Cuisine_Style'].value_counts())

In [13]:
# Создаем списки кухонь, по частоте встречаемости
all_hight_1000 = []
top_top = []
top_mid = []
top_low = []

for i, j in cousins.items():
    if j > 10000:
        all_hight_1000.append(i)
        top_top.append(i)
    elif j < 1000:
        top_low.append(i)
    else:
        all_hight_1000.append(i)
        top_mid.append(i)

In [15]:
# Создаем бинарную переменную наличия у ресторана кухни, с частотой встречаемости более 1 000 
for i in all_hight_1000:
    fin = []
    for j in range(len(data['Cuisine_Style'])):
        if i in data['Cuisine_Style'][j]:
            fin.append(1)
        else:
            fin.append(0)
    data['More_1000'] = fin

In [16]:
# Создаем бинарную переменную наличия у ресторана кухни, с частотой встречаемости более 10 000 
for i in top_top:
    fin = []
    for j in range(len(data['Cuisine_Style'])):
        if i in data['Cuisine_Style'][j]:
            fin.append(1)
        else:
            fin.append(0)
    data['Top_top'] = fin

In [ ]:
    # Создаем бинарную переменную наличия у ресторана кухни, с частотой встречаемости более 1 000 
    for i in all_hight_1000:
        fin = []
        for j in range(len(data['Cuisine_Style'])):
            if i in data['Cuisine_Style'][j]:
                fin.append(1)
            else:
                fin.append(0)
        data['More_1000'] = fin

In [14]:
# определим наиболее часто встречающееся значение и заполним им пропуски
# вынесено в предобработку столбца
pd.DataFrame(cuisine_exploded.Cuisine_Style.value_counts())

,Cuisine_Style
vegetarianfriendly,20472
european,10060
mediterranean,6277
italian,5964
veganoptions,4486
...,...
salvadoran,1
latvian,1
yunnan,1
burmese,1


In [ ]:
# заполним пропуски наиболее часто встречающимся видом кухни (вынесено в предобработку столбца, 
data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: 'vegetarianfriendly' if x[0] == 'nan' else x)

In [17]:
# создадим новый признак - количество предлагаемых в ресторане кухонь
data['Cuisines'] = data['Cuisine_Style'].apply(lambda x: len(x))

In [ ]:
# Поскольку средние значение количества кухонь на ресторан значительно различаются по городам
# отнормируем количество предлагаемых кухонь по среднему значению по городу
data['Cuisines_norm'] = data['Cuisines']/data.City.map(dict(data.groupby(['City'])['Cuisines'].mean()))

### Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города  

In [ ]:
data.loc[:, ['Ranking']].info()
display(pd.DataFrame(data.Ranking.value_counts()))
data.Ranking.describe()

##### пропусков нет, числовой признак, обратим внимание, что это место среди ресторанов своего города, необходимо рассматривать вместе с City

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение

### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

In [22]:
# Нормируем Ranking по среднему значению для города
data['Ranking_norm'] = data['Ranking']/data.City.map(dict(data.groupby(['City'])['Ranking'].mean()))

### 4 Rating — рейтинг ресторана по данным TripAdvisor 

In [ ]:
data.loc[:, ['Rating']].info()
display(pd.DataFrame(data.Rating.value_counts()))
data.Rating.describe()

##### пропусков нет, числовой признак,именно это значение должна будет предсказывать модель, возможные значения от 1 до 5, шаг 0,5

### Price_Range — диапазон цен в ресторане

In [ ]:
data.loc[:, ['Price_Range']].info()
print (pd.DataFrame(data.Price_Range.value_counts())) #display отображает не информативно
data.Price_Range.describe()

##### есть пропуски, ординальный признак, необходима обработка

In [ ]:
# создаем признак, который содержит инфо о пропусках
data['Price_Range_isNAN'] = pd.isna(data['Price_Range']).astype('uint8')

In [23]:
# Создадим словарь для преобразования значений уровня цен
price_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
    
# выполним преобразование
data['Price_Range'].replace(price_dict, inplace=True)
    
# заполним пропущенные значения наиболее часто встречающимся
data['Price_Range'].fillna(data.City.map(dict(data.groupby(['City'])['Price_Range'].mean())), inplace=True)
# Поскольку заполняли средним, то необходимо округлить значение
data['Price_Range'] = data['Price_Range'].apply(round)

###  Number_of_Reviews — количество отзывов о ресторане

In [ ]:
# Проверим наличие пропусков
data['Number_of_Reviews'].isna().value_counts()

In [ ]:
# создаем признак 'Number_of_Reviews_isNAN',который содержит инфо о пропусках
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number_of_Reviews']).astype('uint8')

In [21]:
data.loc[:, ['Number_of_Reviews']].info()
display(pd.DataFrame(data.Number_of_Reviews.value_counts()))
data.Number_of_Reviews.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Number_of_Reviews  40000 non-null  float64
dtypes: float64(1)
memory usage: 312.6 KB


,Number_of_Reviews
125.0,2593
2.0,1916
3.0,1636
4.0,1370
5.0,1181
...,...
1691.0,1
1186.0,1
738.0,1
1721.0,1


count    40000.000000
mean       124.836575
std        286.113295
min          2.000000
25%         10.000000
50%         38.000000
75%        125.000000
max       9660.000000
Name: Number_of_Reviews, dtype: float64

In [ ]:
# Заполним пропуски наиболее часто встречающимся значением
data['Number_of_Reviews'].fillna(data.City.map(dict(data.groupby(['City'])['Number_of_Reviews'].mean())), inplace=True)

In [ ]:
# Создадим признак с информацией о среднем числе отзывов на одного туриста
data['Reviews_per_turist'] = data['Number_of_Reviews']/data['City_Turists']
    
# Поскольку среднее количество отзывов различается по городам
# отнормируем количество отзывов по средне-городскому значению
data['Number_of_Reviews_norm'] = data['Number_of_Reviews']/data.City.map(dict(data.groupby(['City'])['Number_of_Reviews'].

### 7 Reviews — данные о двух отзывах, которые отображаются на сайте ресторана

In [ ]:
data.loc[:, ['Reviews']].info()
display(pd.DataFrame(data.Reviews.value_counts()))
data.Reviews.describe()

In [ ]:
# Создадим переменные содержащие даты первого и второго отзывов
pattern = re.compile('\d\d/\d\d/\d\d\d\d')
data['Reviews'] = data['Reviews'].fillna('')
data['Review_dates'] = data['Reviews'].apply(lambda x: pattern.findall(x))
data['Review_1'] = data['Review_dates'].apply(lambda x: x[:1])
data['Review_2'] = data['Review_dates'].apply(lambda x: x[1:])


# заполняем пропуски в датах самым частым значением для каждого столбца ревью:
for i in range(len(data['Review_1'])):
    if len(data['Review_1'][i]) == 0:
        data['Review_1'][i].append('01/07/2018')
for i in range(len(data['Review_2'])):
    if len(data['Review_2'][i]) == 0:
        data['Review_2'][i].append('01/03/2018')


# переводим в формат Datetime:
data['Review_1'] = data['Review_1'].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y'))
data['Review_2'] = data['Review_2'].apply(lambda x: datetime.strptime(x[0], '%m/%d/%Y'))
data.drop(['Review_dates'], axis=1, inplace=True)

In [ ]:
# Очистим переменную Reviews, сделаем чистые словари для анализа повторных отзывов
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("\\",""))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("\"","'"))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(",",", "))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("',  '",","))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("],  [",","))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(",  "," "))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(", "," "))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("['",""))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("']",""))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("[nan ","nan,"))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("[",""))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("]",""))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("'",""))
data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(" nan",",nan"))
data['Reviews'] = data['Reviews'].apply(lambda x: x.lower())
data['Reviews'] = data['Reviews'].apply(lambda x: x.split(","))

In [25]:
# создаем новый признак 'Review_len' со средний длинной двух отзывов
data['Reviews_len'] = data['Reviews'].apply(lambda x: ((len(x[0])+len(x[1]))/2) if len(x) == 4 else len(x[0]))
# Нормируем по среднему по городу
data['Reviews_len_norm'] = data['Reviews_len']/data.City.map(dict(data.groupby(['City'])['Reviews_len'].mean()))

In [ ]:
# Заполним пропуски средним значением
data['Reviews_len'].replace(to_replace=0, value=data['Reviews_len'].mean(), inplace=True)

In [28]:
# создаем новый признак 'Review_Delta' с разницей в днях между двумя отзывыами из массива:
data['Review_Delta'] = data['Review_1'] - data['Review_2']
data['Review_Delta'] = data['Review_Delta'].apply(lambda x: x.days)

# создаем новый признак 'Review_Delta_now' с разницей в днях между последним отзывом и сегодняшней датой:
data['Review_Delta_now'] = datetime.strptime('02/23/2021', '%m/%d/%Y') - data['Review_1'] 
data['Review_Delta_now'] =  data['Review_Delta_now'].apply(lambda x: x.days)

In [29]:
    # Поскольку некоторые отзывы полностью повторяются
    # cоздадим признак Reviews_repit, отражающий наличие повторных отзывов среди имеющихся у ресторана
    #(повторы на уровне всей базы)

    c = collections.Counter()

    for i in range(len(data['Reviews'])):
        if len(data['Reviews'][i]) == 4:
            c[data['Reviews'][i][0]] += 1
            c[data['Reviews'][i][1]] += 1
        if len(data['Reviews'][i]) == 2 and len(data['Reviews'][i][0]) > 0:
            c[data['Reviews'][i][0]] += 1

    fin = []

    for i in range(len(data['Reviews'])):
        if len(data['Reviews'][i]) == 4:
            if c[data['Reviews'][i][0]] > 1 or c[data['Reviews'][i][1]] > 1:
                fin.append(1)
            else:
                fin.append(0)
        elif len(data['Reviews'][i]) == 2 and len(data['Reviews'][i][0]) > 0:
            if c[data['Reviews'][i][0]] > 1:
                fin.append(1)
            else:
                fin.append(0)
        else:
            fin.append(0)

    data['Reviews_repit'] = fin

##### есть пропуски, категориальный признак, есть повторояющиеся отзывы, возможно использование для дальнейшего улучшения модели

### URL_TA — URL страницы ресторана на TripAdvisor

In [ ]:
data.loc[:, ['URL_TA']].info()
display(pd.DataFrame(data.URL_TA.value_counts()))
data.URL_TA.describe()

##### нет пропусков, категориальный признак, есть повторяющиеся значения, непонятно влияние этого параметра на рейтинг, в текущий момент не используем

### ID_TA — идентификатор ресторана в базе данных TripAdvisor

In [ ]:
data.loc[:, ['ID_TA']].info()
display(pd.DataFrame(data.ID_TA.value_counts()))
data.ID_TA.describe()

##### нет пропусков, категориальный признак, есть повторяющиеся значения, непонятно влияние этого параметра на рейтинг, в текущий момент не используем

### Корреляция признаков
посмотрим, как признаки связаны между собой и с целевой переменной.

In [29]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

KeyError: "['sample'] not found in axis"

сильная положительная корреляция между кол-вом кухонь в ресторане и рейтингом внутри одного города

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [2]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    data = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем ненужные для модели признаки
    data.drop(['ID_TA',], axis = 1, inplace=True)
    # приведем наименование некоторых столбцов к более удобному виду, уберем пробелы
    data = data.rename(columns={'Cuisine Style': 'Cuisine_Style','Price Range': 'Price_Range', 'Number of Reviews': 
                                'Number_of_Reviews'})
    
    # ################### 2. NAN ############################################################## 
    # Обрабатываем пропуски отдельно для каждой переменной
    
    
    # ################### 3. Encoding ############################################################## 
    
    # ##########      3.1 City      ##########
    
    # Создаем дихотомические переменные для каждого города функцией get_dummies
    data['City_gd'] = data['City']
    data = pd.get_dummies(data, columns=['City_gd'], dummy_na=True)
    # так как пропущенных значений нет столбец City_gd_nan удалим
    data = data.drop(['City_gd_nan'], axis = 1)

    # ##########      3.2 Rewie_date      ##########
    
    # Создадим переменные содержащие даты первого и второго отзывов
    pattern = re.compile('\d\d/\d\d/\d\d\d\d')
    data['Reviews'] = data['Reviews'].fillna('')
    data['Review_dates'] = data['Reviews'].apply(lambda x: pattern.findall(x))
    data['Review_1'] = data['Review_dates'].apply(lambda x: x[:1])
    data['Review_2'] = data['Review_dates'].apply(lambda x: x[1:])

    # заполняем пропуски в датах самым частым значением для каждого столбца ревью:
    for i in range(len(data['Review_1'])):
        if len(data['Review_1'][i]) == 0:
            data['Review_1'][i].append('01/07/2018')
    for i in range(len(data['Review_2'])):
        if len(data['Review_2'][i]) == 0:
            data['Review_2'][i].append('01/03/2018')

    # переводим в формат Datetime:
    data['Review_1'] = data['Review_1'].apply(
                       lambda x: datetime.strptime(x[0], '%m/%d/%Y'))
    data['Review_2'] = data['Review_2'].apply(
                       lambda x: datetime.strptime(x[0], '%m/%d/%Y'))
    data.drop(['Review_dates'], axis=1, inplace=True)    

    
    # ##########      3.3 Rewie_text      ##########
    
    # Очистим переменную Reviews, сделаем чистые словари для анализа повторных отзывов
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("\\",""))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("\"","'"))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(",",", "))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("',  '",","))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("],  [",","))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(",  "," "))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(", "," "))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("['",""))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("']",""))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("[nan ","nan,"))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("[",""))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("]",""))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace("'",""))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.replace(" nan",",nan"))
    data['Reviews'] = data['Reviews'].apply(lambda x: x.lower())
    data['Reviews'] = data['Reviews'].apply(lambda x: x.split(","))
    
    
    
    # ################### 4. Feature Engineering ####################################################

    # ##########      4.1 Restaurant_id      ##########

    # Создаем признак уникальности ресторана: "0" - уникальный, "1" - повторяется (предположим, это сеть)
    chain = data.Restaurant_id.value_counts()[data.Restaurant_id.value_counts()>1].index.tolist()
    data['Сhain'] = data[data.Restaurant_id.isin(chain)].Restaurant_id.apply(lambda x: 1)
    data.Сhain = data['Сhain'].fillna(0)


    # ##########      4.2 City      ########## 

    # создаем признак 'City_Num_rest',который содержит информацию о количестве заведений в городе

    cities1 = {'London':5757,
    'Paris':4897,
    'Madrid':3108,
    'Barcelona':2734,
    'Berlin':2155,
    'Milan':2133,
    'Rome':2078,
    'Prague':1443,
    'Lisbon':1300,
    'Vienna':1166,
    'Amsterdam':1086,
    'Brussels':1060,
    'Hamburg':949,
    'Munich':893,
    'Lyon':892,
    'Stockholm':820,
    'Budapest':816,
    'Warsaw':727,
    'Dublin':673,
    'Copenhagen':659,
    'Athens':628,
    'Edinburgh':596,
    'Zurich':538,
    'Oporto':513,
    'Geneva':481,
    'Krakow':443,
    'Oslo':385,
    'Helsinki':376,
    'Bratislava':301,
    'Luxembourg':210,
    'Ljubljana':183}

    data['City_Num_rest'] = data['City'].replace(cities1)


    # создаем признак 'City_Population', который содержит информацию о численности населения города и агломерации
    # (взято с wiki)

    cities2 = {'London':8982000,
    'Paris':2196936,
    'Madrid':6642000,
    'Barcelona':5575000,
    'Berlin':3769000,
    'Milan':1352000,
    'Rome':2873000,
    'Prague':1309000,
    'Lisbon':504718,
    'Vienna':1897000,
    'Amsterdam':821752,
    'Brussels':174383,
    'Hamburg':1845000,
    'Munich':1472000,
    'Lyon':516092,
    'Stockholm':975904,
    'Budapest':1752000,
    'Warsaw':1708000,
    'Dublin':1388000,
    'Copenhagen':602481,
    'Athens':664046,
    'Edinburgh':482005,
    'Zurich':402762,
    'Oporto':214349,
    'Geneva':499480,
    'Krakow':779115,
    'Oslo':681067,
    'Helsinki':631695,
    'Bratislava':424428,
    'Luxembourg':613894,
    'Ljubljana':279631}

    data['City_Population'] = data['City'].replace(cities2)


    # создаем признак 'City_Turists',который содержит информацию о количестве туристов посетивших город по данным Mastercard
    # основной источник: https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors

    cities3 = {'Luxembourg':3000000, 
    'Zurich':2240000, 
    'Geneva':1150000, 
    'Dublin':4970000, 
    'Oslo':4868681, 
    'Amsterdam':8000000, 
    'Copenhagen':1630000, 
    'Stockholm':2080000, 
    'Vienna':6690000, 
    'Helsinki':4000000, 
    'Berlin':4940000, 
    'Hamburg':1450000, 
    'Munich':5250000, 
    'Brussels':2710000, 
    'London':19880000, 
    'Paris':18030000, 
    'Lyon':4868539, 
    'Milan':7650000, 
    'Rome':7120000, 
    'Madrid':5260000, 
    'Barcelona':8200000, 
    'Ljubljana':841320, 
    'Prague':5810000, 
    'Lisbon':3630000, 
    'Oporto':4868539, 
    'Athens':2680000, 
    'Bratislava':1400000, 
    'Budapest':3360000, 
    'Warsaw':1370000, 
    'Krakow':1313277, 
    'Edinburgh':1660000}

    data['City_Turists'] = data['City'].replace(cities3)
    
    
    # создаем признаки 'Rest_on_Pop' и 'Rest_on_Turists',
    # которые содержыт информацию о количестве ресторанов на одного жителя и туриста
    data['Rest_on_Pop'] = data['City_Num_rest']/data['City_Population']
    data['Rest_on_Turists'] = data['City_Num_rest']/data['City_Turists']


    # ##########      4.3 Cuisine_Style      ########## 

    # создаем признак 'Cuisine_Style_isNAN',который содержит инфо о пропусках
    data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine_Style']).astype('uint8')

    # Чистим данные от [] и '',
    # приведем к одному формату (lower, без пробелов),
    # преобразуем в список
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: str(x))
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace("[",""))
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace("]",""))
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace("'",""))
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.replace(" ",""))
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.lower())
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: x.split(","))

    # Создаем словарь ключь - кухня, значение - количество ресторанов с этой кухней
    cuisine_exploded = data.explode('Cuisine_Style')
    cousins = dict(cuisine_exploded['Cuisine_Style'].value_counts())

    # Создаем списки кухонь, по частоте встречаемости
    all_hight_1000 = []
    top_top = []
    top_mid = []
    top_low = []

    for i, j in cousins.items():
        if j > 10000:
            all_hight_1000.append(i)
            top_top.append(i)
        elif j < 1000:
            top_low.append(i)
        else:
            all_hight_1000.append(i)
            top_mid.append(i)

    # Создаем бинарную переменную наличия у ресторана кухни, с частотой встречаемости более 1 000 
    for i in all_hight_1000:
        fin = []
        for j in range(len(data['Cuisine_Style'])):
            if i in data['Cuisine_Style'][j]:
                fin.append(1)
            else:
                fin.append(0)
        data['More_1000'] = fin


    # Создаем бинарную переменную наличия у ресторана кухни, с частотой встречаемости более 10 000 
    for i in top_top:
        fin = []
        for j in range(len(data['Cuisine_Style'])):
            if i in data['Cuisine_Style'][j]:
                fin.append(1)
            else:
                fin.append(0)
        data['Top_top'] = fin

    # заполним пропуски наиболее часто встречающимся видом кухни 
    data['Cuisine_Style'] = data['Cuisine_Style'].apply(lambda x: 'vegetarianfriendly' if x[0] == 'nan' else x)

    # создадим новый признак - количество предлагаемых в ресторане кухонь
    data['Cuisines'] = data['Cuisine_Style'].apply(lambda x: len(x))
    
    # Отнормируем количество предлагаемых кухонь по среднему значению по городу
    data['Cuisines_norm'] = data['Cuisines']/data.City.map(dict(data.groupby(['City'])['Cuisines'].mean()))


    # ##########      4.4 Price_Range      ########## 
    
    # создаем признак, который содержит инфо о пропусках
    data['Price_Range_isNAN'] = pd.isna(data['Price_Range']).astype('uint8')  
  
    # Создадим словарь для преобразования значений уровня цен
    price_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
    
    # выполним преобразование
    data['Price_Range'].replace(price_dict, inplace=True)
    
    # заполним пропущенные значения наиболее часто встречающимся
    data['Price_Range'].fillna(data.City.map(dict(data.groupby(['City'])['Price_Range'].mean())), inplace=True)
    data['Price_Range'] = data['Price_Range'].apply(round)

    

    # ##########      4.5 Number_of_Reviews      ########## 

    # создаем признак 'Number_of_Reviews_isNAN',который содержит инфо о пропусках
    data['Number_of_Reviews_isNAN'] = pd.isna(data['Number_of_Reviews']).astype('uint8')    

    # Заполним пропуски средним значением по городу   
    data['Number_of_Reviews'].fillna(data.City.map(dict(data.groupby(['City'])['Number_of_Reviews'].mean())), inplace=True)
    
    # Создадим признак с информацией о среднем числе отзывов на одного туриста
    data['Reviews_per_turist'] = data['Number_of_Reviews']/data['City_Turists']
    
    # Отнормируем количество отзывов по средне-городскому значению
    data['Number_of_Reviews_norm'] = data['Number_of_Reviews']/data.City.map(dict(data.groupby(['City'])['Number_of_Reviews'].mean()))
    
    
    
    # ##########      4.6 Reviews      ##########    
    
    # создаем новый признак 'Review_len' со средний длинной двух отзывов
    data['Reviews_len'] = data['Reviews'].apply(lambda x: ((len(x[0])+len(x[1]))/2) if len(x) == 4 else len(x[0]))
    
    # Нормируем Review_len по среднему по городу
    data['Reviews_len_norm'] = data['Reviews_len']/data.City.map(dict(data.groupby(['City'])['Reviews_len'].mean()))
    
    # Заполним пропуски средним значением
    data['Reviews_len'].replace(to_replace=0, value=data['Reviews_len'].mean(), inplace=True)

    # создаем новый признак 'Review_Delta' с разницей в днях между двумя отзывыами из массива:
    data['Review_Delta'] = data['Review_1'] - data['Review_2']
    data['Review_Delta'] = data['Review_Delta'].apply(lambda x: x.days)

    # создаем новый признак 'Review_Delta_now' с разницей в днях между последним отзывом и сегодняшней датой:
    data['Review_Delta_now'] = datetime.strptime('02/23/2021', '%m/%d/%Y') - data['Review_1'] 
    data['Review_Delta_now'] =  data['Review_Delta_now'].apply(lambda x: x.days)

    # Создадим признак Reviews_repit, отражающий наличие повторных отзывов среди 2х представленных
    #(повторы на уровне всей базы)
    c = collections.Counter()

    for i in range(len(data['Reviews'])):
        if len(data['Reviews'][i]) == 4:
            c[data['Reviews'][i][0]] += 1
            c[data['Reviews'][i][1]] += 1
        if len(data['Reviews'][i]) == 2 and len(data['Reviews'][i][0]) > 0:
            c[data['Reviews'][i][0]] += 1

    fin = []

    for i in range(len(data['Reviews'])):
        if len(data['Reviews'][i]) == 4:
            if c[data['Reviews'][i][0]] > 1 or c[data['Reviews'][i][1]] > 1:
                fin.append(1)
            else:
                fin.append(0)
        elif len(data['Reviews'][i]) == 2 and len(data['Reviews'][i][0]) > 0:
            if c[data['Reviews'][i][0]] > 1:
                fin.append(1)
            else:
                fin.append(0)
        else:
            fin.append(0)

    data['Reviews_repit'] = fin
    
    # ##########      4.7 Ranking      ##########    
    
    # Нормируем Ranking по среднему по городу 
    data['Ranking_norm'] = data['Ranking']/data.City.map(dict(data.groupby(['City'])['Ranking'].mean()))

    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in data.columns if data[s].dtypes == 'object' or data[s].dtypes == 'datetime64[ns]']
    data.drop(object_columns, axis = 1, inplace=True)
    
    return data

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [30]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [32]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [33]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

NameError: name 'X_train' is not defined

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
